In [61]:
import pandas as pd
import sqlite3
import re
from contextlib import closing
from datetime import datetime

In [62]:
db = "test 2.db"

def get_uid(name:str)->str:
    return re.findall(r"\d+", name)

def get_date(name:int)->datetime:
    return datetime.strptime(
        re.findall(r"\d+", str(name))[0],
        r'%Y%m%d%H%M%S'
    ).strftime(r"%c")

def list_sessions():
    with closing(sqlite3.connect(db)) as conn:
        sessions = pd.read_sql(
            "SELECT id, notes FROM session WHERE (notes is not NULL) and (notes is not \"\")", conn
            )
        sessions["date"] = sessions["id"].apply(get_date)

        return sessions
    
def get_events(uid:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM events_{uid}", conn
        )
    
def get_session(uid:int, start:int, stop:int):
    with closing(sqlite3.connect(db)) as conn:
        return pd.read_sql(
            f"SELECT * FROM session_{uid} LIMIT {stop-start} OFFSET {start}", conn
        )

In [121]:
sessions = list_sessions()
sessions

,id,notes,date
0,20240824222007,unu,Sat Aug 24 22:20:07 2024
1,20240824223239,pruebas,Sat Aug 24 22:32:39 2024
2,20240824223638,nada también,Sat Aug 24 22:36:38 2024
3,20240824223904,prueba,Sat Aug 24 22:39:04 2024
4,20240824225005,sexo,Sat Aug 24 22:50:05 2024
...,...,...,...
79,20241107145351,sofi 1,Thu Nov 7 14:53:51 2024
80,20241107150554,sofi 3,Thu Nov 7 15:05:54 2024
81,20241107151958,miranda5,Thu Nov 7 15:19:58 2024
82,20241130090030,Marcelo,Sat Nov 30 09:00:30 2024


In [94]:
events = get_events(20240912154538)
events.head()

,time,event
0,40,inicio
1,43,buena roja
2,45,buena roja
3,48,buena negra
4,52,buena roja


In [113]:
session = get_session(20240912154538, events.iloc[0, 0], events.iloc[-1,0])
session.loc[session['signal_strength0'] == 0]

,signal_strength0,attention0,meditation0,delta0,theta0,low_alpha0,high_alpha0,low_beta0,high_beta0,low_gamma0,...,attention4,meditation4,delta4,theta4,low_alpha4,high_alpha4,low_beta4,high_beta4,low_gamma4,high_gamma4
0,0,78,43,472956,985869,114796,33722,178058,74203,96801,...,77,47,226843,44148,14252,15816,36112,19548,14177,2641
1,0,61,51,366587,253072,55644,60922,31467,47599,46032,...,77,50,771955,254097,16450,68736,107203,40977,43098,7196
2,0,64,53,313340,35767,10538,17885,47485,23344,10493,...,50,53,203812,151835,19315,30242,9985,14659,7525,3637
3,0,67,35,725162,321383,11337,11394,87468,82721,33143,...,50,53,350538,8715,4383,4286,5119,2143,2188,1481
4,0,84,50,2813945,104284,75080,59170,59888,80387,36977,...,43,43,180557,150085,12964,3428,36791,23942,9680,3093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0,70,57,1901558,94824,106897,55314,57437,85126,18362,...,17,66,799946,139843,21133,40368,12554,24046,21222,14430
145,0,67,64,205712,209657,25431,84456,40013,44480,7289,...,4,51,691632,675799,50349,5685,38116,30611,18843,9084
146,0,61,75,252882,76513,30420,15614,26214,24460,34897,...,23,43,191010,27964,7858,1423,9393,7357,9671,3366
147,0,63,77,271368,202606,52812,58753,22870,32915,16001,...,23,53,133684,67926,26102,3440,12245,6763,4648,5323


In [ ]:
sessions = list_sessions()
size = sessions.index.size
for i in range(size):
    events = get_events(sessions.iloc[i,0])
    with closing(sqlite3.connect(db)) as conn:
        session = get_session(sessions.iloc[i,0], events.iloc[0, 0], events.iloc[-1,0])
        session2 = session.loc[session['signal_strength0'] == 0]
        session2.to_sql(name='datos', con=conn , if_exists='append')

216


IndexError: index 0 is out of bounds for axis 0 with size 0